In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup
import requests, lxml
from re import search
from collections import Counter
import numpy as np
import threading 
import queue
import time

In [2]:
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')
    options.add_argument('--disable-application-cache')
    driver = webdriver.Firefox(options = options)
    return driver

In [3]:
def get_contractor(driver, taxid):
    url = 'https://dauthau.asia/businesslistings/search/?q=' + taxid + '&industry1=&province=0&businesstype=0&lvkd=0'
    driver.get(url)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
    link = soup.find('div', class_= 'listing-summary').a['href']
    driver.get('https://dauthau.asia' + link)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    table = soup.find("table")
    df = pd.read_html(str(table), converters={'Mã số thuế': str, 'Điện thoại công ty': str})[0]
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.iloc[1:2]
    df = df.iloc[:,:17]
    driver.delete_all_cookies()
    return df


In [4]:
def get_contractor_list_driver(driver, tax_id_lst):
    df = get_contractor(driver, tax_id_lst[0])
    for i in tax_id_lst:
        if i != tax_id_lst[0]:
            try:
                df = pd.concat([df,get_contractor(driver, i)])
            except:
                continue
        else:
            pass
    return df

In [5]:
def get_contractor_list(tax_id_lst):
    driver = get_driver()
    df = get_contractor_list_driver(driver, tax_id_lst)
    driver.close()
    return df

In [6]:
# input_file = pd.read_excel("/home/dangpdh/Downloads/dauthau.xlsx", dtype = 'str')

In [6]:
import psycopg2

In [7]:
conn = psycopg2.connect(host="localhost",database="postgres", user="postgres", password="dang1199")

In [8]:
sql = """select distinct "Mã số thuế" from kq_dauthau_202207 kd 
        where "Mã số thuế" not in (select "TAX_ID" from contractor_info ci)"""

In [9]:
input_file = pd.read_sql(sql, conn)

/home/dangpdh/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
input_file = input_file.loc[input_file["Mã số thuế"].str.len() >= 10]

In [11]:
u_lst = list(set(input_file["Mã số thuế"].tolist()))

In [12]:
len(u_lst)

1463

In [13]:
i = 0
while i < 1463:
    print(i)
    start = time.time()
    print(start)
#     lst = u_lst[i:i+1000]
    lst = u_lst
    splits = np.array_split(lst, 2)
    FILE_QUEUE = queue.Queue()
    threads = []
    for n in range(2):
        t = threading.Thread(target = lambda q, arg1: q.put(get_contractor_list(arg1)), args = [FILE_QUEUE, splits[n]])
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    df = pd.DataFrame()
    while not FILE_QUEUE.empty():
        data = FILE_QUEUE.get()
        df = df.append(data, sort=False).reset_index(drop=True)
    df.to_csv("/home/dangpdh/Downloads/contractor_info_" + str(i) +".csv",)
    end = time.time()
    print(end - start)
    i += 1000

0
1660574623.84682


/tmp/ipykernel_8204/3599883610.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, sort=False).reset_index(drop=True)
/tmp/ipykernel_8204/3599883610.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, sort=False).reset_index(drop=True)


2720.043676137924
1000
1660577343.890575


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/home/dangpdh/anaconda3/lib

KeyboardInterrupt: 

    return self.urlopen(method, url, **extra_kw)
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/poolmanager.py", line 376, in urlopen
    response = conn.urlopen(method, u.request_uri, **kw)
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 813, in urlopen
    return self.urlopen(
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 813, in urlopen
    return self.urlopen(
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 813, in urlopen
    return self.urlopen(
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 785, in urlopen
    retries = retries.increment(
  File "/home/dangpdh/anaconda3/lib/python3.9/site-packages/urllib3/util/retry.py", line 592, in increment
    raise MaxRetryError(_pool, url, error or ResponseError(cause))
urllib3.exceptions.MaxRetryError: HTTPConnectionPool(host='localhost',

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 18 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Mã số thuế                                  120 non-null    object
 1   Lượt xem                                    120 non-null    object
 2   Tên công ty                                 120 non-null    object
 3   Tên quốc tế                                 52 non-null     object
 4   Địa chỉ trụ sở                              120 non-null    object
 5   Điện thoại công ty                          120 non-null    object
 6   Fax công ty                                 34 non-null     object
 7   Website công ty                             0 non-null      object
 8   Lĩnh vực kinh doanh                         0 non-null      object
 9   Loại doanh nghiệp                           111 non-null    object
 10  Ngày thành lập công ty    

In [24]:
df.to_csv("/home/dangpdh/Downloads/contractor_info.csv")

In [25]:
requests.get('https://dauthau.asia/businesslistings/search/?q=0941740708&industry1=&province=0&businesstype=0&lvkd=0')

<Response [403]>